<!-- Copyright (c) 2021-2024, InterDigital Communications, Inc
All rights reserved.

Redistribution and use in source and binary forms, with or without 
modification, are permitted (subject to the limitations in the disclaimer 
below) provided that the following conditions are met:

* Redistributions of source code must retain the above copyright notice, 
this list of conditions and the following disclaimer.
* Redistributions in binary form must reproduce the above copyright notice, 
this list of conditions and the following disclaimer in the documentation 
and/or other materials provided with the distribution.
* Neither the name of InterDigital Communications, Inc nor the names of its 
contributors may be used to endorse or promote products derived from this 
software without specific prior written permission.

NO EXPRESS OR IMPLIED LICENSES TO ANY PARTY'S PATENT RIGHTS ARE GRANTED BY 
THIS LICENSE. THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND 
CONTRIBUTORS "AS IS" AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT 
NOT LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A 
PARTICULAR PURPOSE ARE DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDER 
OR CONTRIBUTORS BE LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, 
EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LIMITED TO, 
PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES; LOSS OF USE, DATA, OR PROFITS; 
OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY, 
WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT (INCLUDING NEGLIGENCE OR 
OTHERWISE) ARISING IN ANY WAY OUT OF THE USE OF THIS SOFTWARE, EVEN IF 
ADVISED OF THE POSSIBILITY OF SUCH DAMAGE. -->

# Knowledge Distillation Applied to Auto-Encoder

In [1]:
import io
import math
import numpy as np

from PIL import Image

import torch
import torch.nn as nn
from torchvision import transforms
from torch.utils.data import DataLoader
from pytorch_msssim import ms_ssim

from compressai.datasets import Vimeo90kDataset
from compressai.models.google import ScaleHyperprior

import matplotlib.pyplot as plt
from ipywidgets import interact, widgets

from losses import AverageMeter

In [2]:
def compute_psnr(a, b):
    mse = torch.mean((a - b)**2).item()
    return -10 * math.log10(mse)


def compute_msssim(a, b):
    return ms_ssim(a, b, data_range=1.).item()


def compute_bpp(out_net):
    size = out_net["x_hat"].size()
    num_pixels = size[0] * size[2] * size[3]
    return sum(torch.log(likelihoods).sum() / (-math.log(2) * num_pixels)
               for likelihoods in out_net["likelihoods"].values()).item()

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cpu


In [4]:
N_teacher = 128
N_student = 64
M = 192

## Visualise Models

In [6]:
%%script false --no-raise-error
from torchsummary import summary

# model = ScaleHyperprior(N_teacher, M)
model = ScaleHyperprior(N_student, M)
summary(model, input_size=(3, 256, 256))

In [9]:
%%script false --no-raise-error
import graphviz
from torchview import draw_graph
graphviz.set_jupyter_format("png")

# model = ScaleHyperprior(N_teacher, M)
model = ScaleHyperprior(N_student, M)
model_graph = draw_graph(model, input_size=(1, 3, 256, 256), mode="eval", device="meta")
model_graph.visual_graph

In [13]:
%%script false --no-raise-error
import torch
from torchviz import make_dot

# model = ScaleHyperprior(N_teacher, M)
model = ScaleHyperprior(N_student, M)
x = torch.randn(1, 3, 256, 256)
out = model(x)

make_dot(out["x_hat"].mean(), params=dict(model.named_parameters()))
# make_dot(y.mean(), params=dict(model.named_parameters()), show_attrs=True, show_saved=True)

## Load Models

In [ ]:
# Teacher Model
id = 254451
teacher_net = ScaleHyperprior(N_teacher, M)
checkpoint = torch.load(f"train_res/{id}/checkpoint_best.pth.tar",
    weights_only=True, map_location=torch.device("cpu"))
teacher_net.load_state_dict(checkpoint["state_dict"])
teacher_net = teacher_net.eval().to(device)

print(f"Parameters: {sum(p.numel() for p in teacher_net.parameters())}")

In [ ]:
# Student Model
id = 254452
student_net = ScaleHyperprior(N_student, M)
checkpoint = torch.load(f"train_res/{id}/checkpoint_best.pth.tar",
    weights_only=True, map_location=torch.device("cpu"))
student_net.load_state_dict(checkpoint["state_dict"])
student_net = student_net.eval().to(device)

print(f"Parameters: {sum(p.numel() for p in student_net.parameters())}")

In [ ]:
# Student Model with KD (no latent loss)
id = 257451
student_kd_net_1 = ScaleHyperprior(N_student, M)
checkpoint = torch.load(f"train_res/{id}/checkpoint_best.pth.tar",
    weights_only=True, map_location=torch.device("cpu"))
student_kd_net_1.load_state_dict(checkpoint["state_dict"])
student_kd_net_1 = student_kd_net_1.eval().to(device)

print(f"Parameters: {sum(p.numel() for p in student_kd_net_1.parameters())}")

In [ ]:
# Student Model with KD (latent loss)
id = 257452
student_kd_net_2 = ScaleHyperprior(N_student, M)
checkpoint = torch.load(f"train_res/{id}/checkpoint_best.pth.tar",
    weights_only=True, map_location=torch.device("cpu"))
student_kd_net_2.load_state_dict(checkpoint["state_dict"])
student_kd_net_2 = student_kd_net_2.eval().to(device)

print(f"Parameters: {sum(p.numel() for p in student_kd_net_2.parameters())}")

In [9]:
models = {
    "teacher": {"model": teacher_net, "metrics": {}},
    "student": {"model": student_net, "metrics": {}},
    "student_kd_1": {"model": student_kd_net_1, "metrics": {}},
    "student_kd_2": {"model": student_kd_net_2, "metrics": {}},
}

## Models Comparison

### Saint Malo

In [ ]:
test_transforms = transforms.Compose(
    [transforms.RandomCrop((256,256)), transforms.ToTensor()]
)

img = Image.open("../data/assets/stmalo_fracape.png").convert("RGB")
x = test_transforms(img).unsqueeze(0).to(device)
print(x.shape)

In [33]:
# Inference
criterion = nn.MSELoss()
with torch.no_grad():
    for model in models.keys():
        output = models[model]["model"].forward(x)

        models[model]["metrics"]["mse"] = criterion(output["x_hat"], x).item()
        models[model]["metrics"]["psnr"] = compute_psnr(x, output["x_hat"])
        models[model]["metrics"]["ms_ssim"] = compute_msssim(x, output["x_hat"])
        models[model]["metrics"]["bpp"] = compute_bpp(output)

        models[model]["output"] = output

In [ ]:
fig, axs = plt.subplots(4, 4, figsize=(16, 12))

orig = transforms.ToPILImage()(x.squeeze().cpu())

for ax in axs.ravel():
    ax.axis("off")

for i, model in enumerate(models.keys()):
    axs[i, 0].imshow(orig)
    axs[i, 0].title.set_text("Original")

    recons = transforms.ToPILImage()(models[model]["output"]["x_hat"].squeeze().cpu())
    axs[i, 1].imshow(recons)
    axs[i, 1].title.set_text("Reconstructed")

    diffs = torch.mean((models[model]["output"]["x_hat"] - x).abs(), axis=1).squeeze().cpu()
    axs[i, 2].imshow(diffs, cmap="viridis")
    axs[i, 2].title.set_text(f"Difference")

    axs[i, 3].set_axis_off()
    axs[i, 3].table(rowLabels=list(models[model]["metrics"].keys()),
                    cellText=[[m] for m in models[model]["metrics"].values()],
                    loc="center")

plt.show()